In [1]:
from pybullet_environment import *
from guide import *

pybullet build time: Jul 17 2023 13:38:55


In [2]:
env = RobotEnvironment(manipulator = True)
guide = IntersectionVolumeGuide(env)

argv[0]=
startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=3
argv[0] = --unused
argv[1] = 
argv[2] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce GTX 1660 Ti/PCIe/SSE2
GL_VERSION=3.3.0 NVIDIA 535.86.05
GL_SHADING_LANGUAGE_VERSION=3.30 NVIDIA via Cg compiler
pthread_getconcurrency()=0
Version = 3.3.0 NVIDIA 535.86.05
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce GTX 1660 Ti/PCIe/SSE2
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started


In [16]:
traj = np.load("traj.npy")
start_joints = traj[0, :]
goal_joints = traj[-1, :]

In [22]:
goal_joints[0] = 0.55
print(goal_joints)

[ 0.55        0.5573511  -0.1629493  -1.9912002  -0.23168434  2.214461
 -0.38091224]


In [24]:
joint_config = np.zeros((9,))[:-2]
joint_config[0] = -0.371
joint_config[1] = 0.522
joint_config[2] = -0.134
joint_config[3] = -1.4
joint_config[4] = 0.341
joint_config[5] = 1.7
joint_config[6] = 0.572

for i, joint_ind in enumerate(env.joints):
    env.client_id.resetJointState(env.manipulator, joint_ind, joint_config[i])

In [13]:
for i, joint_ind in enumerate(env.joints):
    env.client_id.resetJointState(env.manipulator, joint_ind, start_joints[i])

In [25]:
for i, joint_ind in enumerate(env.joints):
    env.client_id.resetJointState(env.manipulator, joint_ind, goal_joints[i])

In [11]:
obs_pos = np.array([0.6, 0., 0.24])
obs_ori = np.array([0., 0., 0.])
obs_quat = np.array(env.client_id.getQuaternionFromEuler(obs_ori))
obs_pose = np.array([*obs_pos, *obs_quat])

obs_size = np.array([0.5, 0.06, 0.48])

vuid = env.client_id.createVisualShape(p.GEOM_BOX, 
                                       halfExtents = obs_size/2,
                                       rgbaColor = np.hstack([env.colors['yellow'], np.array([1.0])]))

obs_id = env.client_id.createMultiBody(baseVisualShapeIndex = vuid, 
                                       basePosition = obs_pos, 
                                       baseOrientation = obs_quat)

In [10]:
env.client_id.removeBody(obs_id)

In [9]:
joint_config = np.zeros((9,))[:-2]
joint_config[0] = 0.11
joint_config[1] = 0.913
joint_config[2] = 0.12
joint_config[3] = -0.96
joint_config[4] = 0.23
joint_config[5] = 1.43
joint_config[6] = 0.3

for i, joint_ind in enumerate(env.joints):
    env.client_id.resetJointState(env.manipulator, joint_ind, joint_config[i])

In [10]:
joint_tensor = torch.tensor(joint_config, dtype = torch.float32)
joint_tensor.requires_grad = True

In [12]:
for _ in range(100):

    joint_tensor.grad = None
    cost = guide.cost(joint_tensor, obs_pose, obs_size)

    cost.backward()

    with torch.no_grad():
        joint_tensor -= 0.2 * joint_tensor.grad

    # joint_config = joint_tensor.detach().numpy()
    # for i, joint_ind in enumerate(env.joints):
    #     env.client_id.resetJointState(env.manipulator, joint_ind, joint_config[i])

: 

In [9]:
print(joint_tensor.grad)

tensor([-0.0462,  0.0962, -0.0306, -0.0592, -0.0029, -0.0081, -0.0001])


: 